In [39]:
# import requests
#https://ca.global.nba.com/statistics/
page = requests.get("https://ca.global.nba.com/statistics/")

from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
#

blocs = set()
for bloc in soup.find_all('div', class_='sib-table-full-player-stats'):
        print(bloc)


<div class="sib-table-container sib-table-full-player-stats sib-league-player-stats players-stats" data-ng-cloak="" data-ng-controller="FullPlayerStatsController">
<div class="table-title">
<div class="title">
<span>
<span class="table-title__season-wrapper">
<span class="season-type {{ seasonType }}" ng-if="seasonType">{{ seasonYearDisplay + " " + ('_seasonType.' + seasonType | i18next) }}</span>
</span>
<span class="full-player-stats__title">{{ 'fullTeamStats.title' | i18next }}</span>
</span>
</div>
<div class="filter-button" data-ng-class="{toggled: showFilters==true}" data-ng-click="showFilters=!showFilters">
<i class="icon icon-remove" data-ng-show="showFilters==true"></i>
<i class="icon icon-cog" data-ng-show="showFilters==false"></i>
</div>
<div class="sib-table-filters" data-ng-show="showFilters==true">
<div class="filter-row stats-filters">
<div class="filters">
<select data-ng-model="selectedFilters.seasonType" data-ng-options="c.display for c in filterData.seasonType" id="f

In [10]:
#Import de la librairie pandas
import pandas

#Lecture des fichiers de données en spécifiant le séparateur point-virgule ';'
equipeJO = pandas.read_csv("20162020_JO_Team.csv", sep=';')

joueurJO2016 = pandas.read_csv("2016_JO_Player.csv", sep=';')
joueurNBA2016 = pandas.read_csv("2016_NBA_Player.csv", sep=';')

#Liste provenant de WashingtonPost. Information peut-être redondante avec le non-WP. 
joueurNBAWP2016 = pandas.read_csv("2016_NBA_Player_WP.csv", sep=';')
equipeNBA2016 = pandas.read_csv("2016_NBA_Team.csv", sep=';')

#2020
joueurJO2020 = pandas.read_csv("2020_JO_Player.csv", sep=';')
joueurNBA2020 = pandas.read_csv("2020_NBA_Player.csv", sep=';')
equipeNBA2020 = pandas.read_csv("2020_NBA_Team.csv", sep=';')

#print(notes.dtypes)
#joueurNBA2016

Quel est la corrélation entre la participation aux Olympiques et le classements des joueurs dans la NBA?

In [40]:
# jointure
#df_sales_full = df_sales.join(df_product.set_index("product_id"), on="product_id")

#print(joueurJO2016.columns)
joueur2016 = joueurJO2016.join(joueurNBA2016.set_index('Name'), on="Name", lsuffix='JO', rsuffix='NBA', how='inner')
joueur2020 = joueurJO2020.join(joueurNBA2020.set_index('Name'), on="Name", lsuffix='JO', rsuffix='NBA', how='inner')


#print(joueurJO2016.dtypes)
#print(joueurNBA2016.dtypes)
print(joueur2020.dtypes)


Name                                object
AgeJO                                int64
TeamJO                              object
PositionJO                          object
Games PlayedJO                       int64
Game StartedJO                       int64
Field Goals Made                     int64
 Field Goals Attempted               int64
Field Goals Made PercentageJO      float64
3 Point Made                         int64
3 Point Attempted                    int64
3 Point Made Percentage            float64
Free Throws Made                     int64
Free Throws Attempted                int64
Free Throws Made PercentageJO      float64
Total Rebounds                       int64
Rebounds Average                   float64
Total Points                         int64
Points Average                     float64
Total Assists                        int64
Assists Average                    float64
Total Blocks                         int64
Blocks Average                     float64
Steals     

Est-ce que les joueurs de la NBA ont une performance similaire moyenne en saison régulière comparé aux olympiques (Points, rebound, assist et block)?

Est-ce que la performance moyenne d'une équipe de la NBA est semblable à une équipe Olympique?

Est-ce que l'écart-type est similaire entre les joueurs d'une équipe de la NBA et ceux d'une équipe Olympique?